In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend
import tensorflow
tensorflow.set_random_seed(7)
from sklearn.model_selection import train_test_split
import numpy
numpy.random.seed(7)

Using TensorFlow backend.


## Restricting GPU memory usage

The code here should be added to any work you do on Volta.  If you don't, then your code will monopolize all available memory on each of the 4 GPUs on the machine, preventing others from working on it.  If you do **that**, you will be frowned upon.

The code in the next cell has the effect that:
1. Memory use will start off with some small fraction of the memory on each GPU.
1. It will grow if necessary (since `allow_growth` is set to `True`).
1. It will max out at 5% of overall memory.  Given the GPUs we have, this gives you (4 x 808 MB), which should be sufficient here.

In [2]:
########################
# Limit TensorFlow GPU use.
config = tensorflow.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.05
backend.tensorflow_backend.set_session(tensorflow.Session(config=config))
########################

## Loading our data

The function `train_test_split` comes from `sklearn` (from `SciKit`). Two things to note:
1. Setting `test_size` to `0.25` means that 25% of the data is held aside as testing data.  
1. Randomness here can affect overall results.

In [3]:
dataset = numpy.loadtxt("diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.25)

## Building and evaluating our model

The initial model is a simple network, with two hidden layers, each of which uses the RELU function for activation.  There is also a simple single-class output neuron using the Sigmoid logistic function.  The loss function is binary cross entropy (and so we want to measure the associated binary accuracy value).  The model is trained using a batch size equal to all of the training data, for 1,000 epochs.

For this network, do the following:
1. In binary classification, **cross entropy** measures the error on an output probability value between 0 and 1, and can be calculated as: $−(ylog(p)+(1−y)log(1−p))$, such that the loss increases as the predicted value diverges from the actual output. The binary cross entropy function takes into account the proximity of a prediction to the actual value (as opposed to simply correct/incorrect) and is a more granular way to compute error.
1. Epochs are essentially the number of passes over our dataset that the algorithm will take before it terminates learning. A table of results from running the model with specified epochs is given below. We observe that training accuracy steadily increases up to 10,000 epoch, indicating that the model benefits in updating weights and minimizing loss as we make more passes over the data. However, after 1,000 epoch we start to see the test accuracy decreasing, which is an indication of overfitting and that we have trained the model to fit the training data too accurately. We seek to find a optimal balance between these two accuracies.

| Epochs | Train Acc. | Test Acc. |
| ------ | ---------- | --------- |
| 500 | 65.97% | 63.54% |
| 1000 | 78.82% | 76.04% |
| 2000 | 83.16% | 74.48% |
| 5000 | 92.53% | 71.35% |
| 10000 | 90.80% | 71.35% |

1. Dropout layers randomly select a proportion of neurons to "ignore" during training, and is commonly used as a method to avoid overfitting due to essentially forcing other neurons to pick up the "slack" (gain additional information) from the neurons that were dropped. In testing multiple dropout layers and monitoring their effects on training and test accuracy, we see that training accuracy is maximized in our original model (0 dropout layers), and steadily decreases as we increase dropout proportions. We also note that adding dropouts later in the network generally perform better for our model, though still not as well as with no dropout. On the other hand, we actually see a 2% increase in test accuracy from our original model when adding a single dropout layer after each of the 25-neuron dense layers. Aside from this, the test error essentially follows the same trend that we saw in test accuracy.

| After Layer 1 | After Layer 2 | Train Acc. | Test Acc. |
| ------------- | ------------- | ---------- | --------- |
| 0% | 0% | 78.82% | 76.04% |
| 0% | 10% | 78.30% | 70.83% |
| 0% | 20% | 77.43% | 78.12% |
| 10% | 0% | 74.13% | 69.27% |
| 10% | 10% | 74.48% | 72.92% |
| 20% | 20% | 69.62% | 64.58% |
| 50% | 50% | 65.62% | 63.54% |
| 50% | 20% | 66.67% | 64.06% |
| 20% | 50% | 70.31% | 65.10% |

In [40]:
# create model
model = Sequential()
model.add(Dense(25, input_dim=8, activation='relu'))
#model.add(Dropout(0.1)) # added
model.add(Dense(25, activation='relu'))
#model.add(Dropout(0.1)) # added
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# Fit the model
model.fit(train_x, train_y, epochs=1000, batch_size=len(train_x), validation_data=(test_x, test_y), verbose=1)

Train on 576 samples, validate on 192 samples
Epoch 1/1000
576/576 [==============================] - 1s 2ms/step - loss: 7.5421 - binary_accuracy: 0.3524 - val_loss: 6.7058 - val_binary_accuracy: 0.3646
Epoch 2/1000
576/576 [==============================] - 0s 17us/step - loss: 7.0955 - binary_accuracy: 0.3663 - val_loss: 5.8151 - val_binary_accuracy: 0.3646
Epoch 3/1000
576/576 [==============================] - 0s 15us/step - loss: 6.4354 - binary_accuracy: 0.3576 - val_loss: 4.8381 - val_binary_accuracy: 0.3646
Epoch 4/1000
576/576 [==============================] - 0s 14us/step - loss: 5.4602 - binary_accuracy: 0.3733 - val_loss: 3.8726 - val_binary_accuracy: 0.4115
Epoch 5/1000
576/576 [==============================] - 0s 14us/step - loss: 4.7450 - binary_accuracy: 0.3958 - val_loss: 3.0214 - val_binary_accuracy: 0.4479
Epoch 6/1000
576/576 [==============================] - 0s 13us/step - loss: 4.2256 - binary_accuracy: 0.3837 - val_loss: 2.4167 - val_binary_accuracy: 0.4844
E

Epoch 52/1000
576/576 [==============================] - 0s 15us/step - loss: 1.4306 - binary_accuracy: 0.5139 - val_loss: 0.8450 - val_binary_accuracy: 0.5260
Epoch 53/1000
576/576 [==============================] - 0s 14us/step - loss: 1.6246 - binary_accuracy: 0.5122 - val_loss: 0.8259 - val_binary_accuracy: 0.5312
Epoch 54/1000
576/576 [==============================] - 0s 14us/step - loss: 1.4960 - binary_accuracy: 0.5434 - val_loss: 0.8042 - val_binary_accuracy: 0.5573
Epoch 55/1000
576/576 [==============================] - 0s 15us/step - loss: 1.6358 - binary_accuracy: 0.5278 - val_loss: 0.7827 - val_binary_accuracy: 0.5833
Epoch 56/1000
576/576 [==============================] - 0s 14us/step - loss: 1.3944 - binary_accuracy: 0.5486 - val_loss: 0.7672 - val_binary_accuracy: 0.6094
Epoch 57/1000
576/576 [==============================] - 0s 14us/step - loss: 1.4216 - binary_accuracy: 0.5295 - val_loss: 0.7617 - val_binary_accuracy: 0.6302
Epoch 58/1000
576/576 [=================

Epoch 104/1000
576/576 [==============================] - 0s 15us/step - loss: 0.8912 - binary_accuracy: 0.6233 - val_loss: 0.6631 - val_binary_accuracy: 0.6771
Epoch 105/1000
576/576 [==============================] - 0s 15us/step - loss: 0.9454 - binary_accuracy: 0.6094 - val_loss: 0.6645 - val_binary_accuracy: 0.6719
Epoch 106/1000
576/576 [==============================] - 0s 15us/step - loss: 0.9534 - binary_accuracy: 0.5938 - val_loss: 0.6639 - val_binary_accuracy: 0.6667
Epoch 107/1000
576/576 [==============================] - 0s 14us/step - loss: 1.0243 - binary_accuracy: 0.5851 - val_loss: 0.6618 - val_binary_accuracy: 0.6719
Epoch 108/1000
576/576 [==============================] - 0s 15us/step - loss: 0.9475 - binary_accuracy: 0.6076 - val_loss: 0.6591 - val_binary_accuracy: 0.6771
Epoch 109/1000
576/576 [==============================] - 0s 13us/step - loss: 0.9478 - binary_accuracy: 0.5972 - val_loss: 0.6551 - val_binary_accuracy: 0.6875
Epoch 110/1000
576/576 [==========

Epoch 155/1000
576/576 [==============================] - 0s 16us/step - loss: 0.8222 - binary_accuracy: 0.6267 - val_loss: 0.6542 - val_binary_accuracy: 0.6667
Epoch 156/1000
576/576 [==============================] - 0s 14us/step - loss: 0.7659 - binary_accuracy: 0.6597 - val_loss: 0.6531 - val_binary_accuracy: 0.6667
Epoch 157/1000
576/576 [==============================] - 0s 14us/step - loss: 0.7391 - binary_accuracy: 0.6458 - val_loss: 0.6513 - val_binary_accuracy: 0.6823
Epoch 158/1000
576/576 [==============================] - 0s 14us/step - loss: 0.7298 - binary_accuracy: 0.6441 - val_loss: 0.6506 - val_binary_accuracy: 0.6771
Epoch 159/1000
576/576 [==============================] - 0s 14us/step - loss: 0.7157 - binary_accuracy: 0.6632 - val_loss: 0.6506 - val_binary_accuracy: 0.6667
Epoch 160/1000
576/576 [==============================] - 0s 13us/step - loss: 0.7838 - binary_accuracy: 0.6597 - val_loss: 0.6515 - val_binary_accuracy: 0.6615
Epoch 161/1000
576/576 [==========

Epoch 206/1000
576/576 [==============================] - 0s 16us/step - loss: 0.7321 - binary_accuracy: 0.6510 - val_loss: 0.6471 - val_binary_accuracy: 0.6719
Epoch 207/1000
576/576 [==============================] - 0s 17us/step - loss: 0.7201 - binary_accuracy: 0.6319 - val_loss: 0.6472 - val_binary_accuracy: 0.6458
Epoch 208/1000
576/576 [==============================] - 0s 16us/step - loss: 0.7325 - binary_accuracy: 0.6354 - val_loss: 0.6478 - val_binary_accuracy: 0.6562
Epoch 209/1000
576/576 [==============================] - 0s 16us/step - loss: 0.7339 - binary_accuracy: 0.6302 - val_loss: 0.6496 - val_binary_accuracy: 0.6458
Epoch 210/1000
576/576 [==============================] - 0s 16us/step - loss: 0.6898 - binary_accuracy: 0.6597 - val_loss: 0.6509 - val_binary_accuracy: 0.6510
Epoch 211/1000
576/576 [==============================] - 0s 19us/step - loss: 0.6837 - binary_accuracy: 0.6441 - val_loss: 0.6510 - val_binary_accuracy: 0.6562
Epoch 212/1000
576/576 [==========

...

Epoch 767/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5404 - binary_accuracy: 0.7118 - val_loss: 0.5974 - val_binary_accuracy: 0.6875
Epoch 768/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5716 - binary_accuracy: 0.7101 - val_loss: 0.5974 - val_binary_accuracy: 0.6875
Epoch 769/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5685 - binary_accuracy: 0.6962 - val_loss: 0.5982 - val_binary_accuracy: 0.6875
Epoch 770/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5558 - binary_accuracy: 0.6979 - val_loss: 0.5999 - val_binary_accuracy: 0.6875
Epoch 771/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5433 - binary_accuracy: 0.7240 - val_loss: 0.6016 - val_binary_accuracy: 0.6823
Epoch 772/1000
576/576 [==============================] - 0s 13us/step - loss: 0.5416 - binary_accuracy: 0.7049 - val_loss: 0.6020 - val_binary_accuracy: 0.6823
Epoch 773/1000
576/576 [==========

Epoch 818/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5304 - binary_accuracy: 0.7257 - val_loss: 0.5971 - val_binary_accuracy: 0.6927
Epoch 819/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5268 - binary_accuracy: 0.7083 - val_loss: 0.5976 - val_binary_accuracy: 0.6875
Epoch 820/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5586 - binary_accuracy: 0.7014 - val_loss: 0.5990 - val_binary_accuracy: 0.6823
Epoch 821/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5610 - binary_accuracy: 0.7083 - val_loss: 0.5995 - val_binary_accuracy: 0.6823
Epoch 822/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5394 - binary_accuracy: 0.7309 - val_loss: 0.6005 - val_binary_accuracy: 0.6823
Epoch 823/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5356 - binary_accuracy: 0.7274 - val_loss: 0.6004 - val_binary_accuracy: 0.6771
Epoch 824/1000
576/576 [==========

Epoch 869/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5342 - binary_accuracy: 0.7240 - val_loss: 0.5958 - val_binary_accuracy: 0.6823
Epoch 870/1000
576/576 [==============================] - 0s 16us/step - loss: 0.5281 - binary_accuracy: 0.7222 - val_loss: 0.5943 - val_binary_accuracy: 0.6823
Epoch 871/1000
576/576 [==============================] - 0s 17us/step - loss: 0.5376 - binary_accuracy: 0.7326 - val_loss: 0.5930 - val_binary_accuracy: 0.6875
Epoch 872/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5294 - binary_accuracy: 0.7170 - val_loss: 0.5925 - val_binary_accuracy: 0.6875
Epoch 873/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5163 - binary_accuracy: 0.7431 - val_loss: 0.5934 - val_binary_accuracy: 0.6823
Epoch 874/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5503 - binary_accuracy: 0.7066 - val_loss: 0.5947 - val_binary_accuracy: 0.6823
Epoch 875/1000
576/576 [==========

Epoch 920/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5233 - binary_accuracy: 0.7378 - val_loss: 0.5917 - val_binary_accuracy: 0.6927
Epoch 921/1000
576/576 [==============================] - 0s 16us/step - loss: 0.5197 - binary_accuracy: 0.7396 - val_loss: 0.5890 - val_binary_accuracy: 0.6979
Epoch 922/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5390 - binary_accuracy: 0.7309 - val_loss: 0.5875 - val_binary_accuracy: 0.6875
Epoch 923/1000
576/576 [==============================] - 0s 14us/step - loss: 0.5489 - binary_accuracy: 0.7153 - val_loss: 0.5876 - val_binary_accuracy: 0.6927
Epoch 924/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5156 - binary_accuracy: 0.7361 - val_loss: 0.5890 - val_binary_accuracy: 0.6979
Epoch 925/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5433 - binary_accuracy: 0.7292 - val_loss: 0.5900 - val_binary_accuracy: 0.6927
Epoch 926/1000
576/576 [==========

Epoch 971/1000
576/576 [==============================] - 0s 13us/step - loss: 0.5352 - binary_accuracy: 0.7274 - val_loss: 0.5885 - val_binary_accuracy: 0.6875
Epoch 972/1000
576/576 [==============================] - 0s 13us/step - loss: 0.5045 - binary_accuracy: 0.7431 - val_loss: 0.5873 - val_binary_accuracy: 0.6979
Epoch 973/1000
576/576 [==============================] - 0s 12us/step - loss: 0.5233 - binary_accuracy: 0.7378 - val_loss: 0.5863 - val_binary_accuracy: 0.6927
Epoch 974/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5342 - binary_accuracy: 0.7170 - val_loss: 0.5858 - val_binary_accuracy: 0.6979
Epoch 975/1000
576/576 [==============================] - 0s 15us/step - loss: 0.5138 - binary_accuracy: 0.7361 - val_loss: 0.5857 - val_binary_accuracy: 0.6979
Epoch 976/1000
576/576 [==============================] - 0s 12us/step - loss: 0.5259 - binary_accuracy: 0.7309 - val_loss: 0.5849 - val_binary_accuracy: 0.6979
Epoch 977/1000
576/576 [==========

## Evaluation of final model on training data

In [41]:
# evaluate the model
scores = model.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

576/576 [==============================] - 0s 74us/step

binary_accuracy: 74.13%


## Evaluation of final model on testing data

In [42]:
# evaluate the model
scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 84us/step

binary_accuracy: 69.27%
